

**asyncio**:	Concorrente (assíncrona, 1 thread)	I/O lento e muitas conexões simultâneas	Não aproveita múltiplos núcleos da CPU  
**threading**:	Concorrente (múltiplas threads)	I/O rápido, tarefas leves simultâneas	GIL impede paralelismo real em tarefas de CPU  
**multiprocessing**:	Paralelo (múltiplos processos)	Tarefas pesadas de CPU	Mais consumo de memória e overhead de comunicação  

## asyncio  
Ideal para servidores web, chamadas de API, scraping de sites com muitas requisições simultâneas.  

Exemplo: um servidor que precisa lidar com milhares de conexões sem bloquear.

## threading  
Bom para tarefas que envolvem I/O rápido, como leitura de arquivos, pequenas requisições de rede.  Mas sofre em tarefas de CPU por causa do GIL.

Exemplo: baixar várias imagens da internet ao mesmo tempo.

## multiprocessing  
Perfeito para tarefas que exigem muito processamento, como cálculos matemáticos intensos ou processamento de imagens.  Usa realmente múltiplos núcleos (cada um com seu GIL).

Exemplo: aplicar filtros em milhares de fotos simultaneamente.  

## Global Interpreter Lock (GIL) 

é um mecanismo interno do interpretador padrão do Python (CPython) que impede que múltiplas threads executem código Python ao mesmo tempo dentro de um único processo2.

🔐 Por que o GIL existe?
O principal motivo é garantir segurança na manipulação de memória. O Python usa contadores de referência para gerenciar objetos, e esse sistema não é thread-safe por padrão. O GIL funciona como um mutex (bloqueio de exclusão mútua) que protege essas operações internas, evitando que duas threads alterem estruturas de dados ao mesmo tempo e causem corrupção ou bugs difíceis de rastrear.

⚠️ Implicações práticas
Mesmo que você crie várias threads com threading, apenas uma pode executar código Python por vez.

Isso significa que não há paralelismo real para tarefas que exigem muito da CPU.

O GIL não afeta operações de I/O (como leitura de arquivos ou chamadas de rede), onde o tempo de espera pode ser aproveitado por outras threads.

In [3]:
import asyncio

async def tarefa(nome, tempo):
    print(f"{nome} começou")
    await asyncio.sleep(tempo)
    print(f"{nome} terminou após {tempo}s")

async def main():
    await asyncio.gather(
        tarefa("Tarefa 1", 2),
        tarefa("Tarefa 2", 1),
        tarefa("Tarefa 3", 3)
    )

await main()


Tarefa 1 começou
Tarefa 2 começou
Tarefa 3 começou
Tarefa 2 terminou após 1s
Tarefa 1 terminou após 2s
Tarefa 3 terminou após 3s


In [4]:
import threading
import time

def tarefa(nome, tempo):
    print(f"{nome} começou")
    time.sleep(tempo)
    print(f"{nome} terminou após {tempo}s")

threads = []
for i in range(3):
    t = threading.Thread(target=tarefa, args=(f"Tarefa {i+1}", i+1))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


Tarefa 1 começou
Tarefa 2 começou
Tarefa 3 começou
Tarefa 1 terminou após 1s
Tarefa 2 terminou após 2s
Tarefa 3 terminou após 3s


In [8]:
from concurrent.futures import ThreadPoolExecutor
import time

def tarefa(nome, tempo):
    inicio = time.time()
    time.sleep(tempo)
    fim = time.time()
    return f"{nome} terminou após {tempo}s (em {fim - inicio:.2f}s)"

nomes_tempos = [("Tarefa 1", 2), ("Tarefa 2", 3), ("Tarefa 3", 1)]

with ThreadPoolExecutor() as executor:
    resultados = executor.map(lambda nt: tarefa(*nt), nomes_tempos)

for resultado in resultados:
    print(resultado)


Tarefa 1 terminou após 2s (em 2.00s)
Tarefa 2 terminou após 3s (em 3.00s)
Tarefa 3 terminou após 1s (em 1.00s)
